In [2]:
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import openai
import random
from retrying import retry
import pyarrow as pa
import pyarrow.parquet as pq
from transformers import AutoTokenizer

openai.api_base = "https://one.aiskt.com/v1"


class OpenAIGPT:
    def __init__(self, model_name="gpt-3.5-turbo", keys_path=None):
        self.model_name = model_name
        with open(keys_path, encoding="utf-8", mode="r") as fr:
            self.keys = [line.strip() for line in fr if len(line.strip()) >= 4]

    def __post_process(self, response):
        return response["choices"][0]["message"]["content"]

    @retry(wait_fixed=300, stop_max_attempt_number=50)
    def __call__(self, message):
        if message is None or message == "":
            return False, "Your input is empty."

        # current_key = random.choice(self.keys)
        current_key = self.keys[0] if len(self.keys) == 1 else random.choice(self.keys)
        openai.api_key = current_key
        response = openai.ChatCompletion.create(
            model=self.model_name,
            messages=[{"role": "user", "content": message}],
            temperature=0.3,
            top_p=0.1,
            frequency_penalty=0.6,
            presence_penalty=0.6,
            n=1,
        )
        return self.__post_process(response)


igpt = OpenAIGPT(keys_path="apikeys.txt")
model_name = "/home/zhangmin/.cache/modelscope/hub/qwen/Qwen-7B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name,  trust_remote_code=True)

In [3]:
igpt('hello')

'Hello! How can I assist you today?'

In [4]:
import pandas as pd

data = pd.read_parquet('../data/announcement/600036.SH.parquet')
data.head(1)

,ann_date,ts_code,name,title,url,rec_time,content
0,20240326,600036.SH,招商银行,招商银行股份有限公司董事会审计委员会对2023年度会计师事务所履职情况的评估报告,http://dataclouds.cninfo.com.cn/shgonggao/2024...,2024-03-25 18:43:02,时间：2024-03-25 18:43:02\n标题：招商银行股份有限公司董事会审计委员会对...


In [5]:
def process_row(row):

    def calculate_token_length(text):
        global tokenizer
        tokens = tokenizer.encode(text, truncation=True)
        return len(tokens)

    def predict(model, prompt):
        # igpt = OpenAIGPT(keys_path="apikeys.txt")
        global igpt
        try:
            return (igpt(prompt))
        except openai.error.APIError as e:
            print(f"Error occurred with prompt: {prompt}")
            print(f"Error: {e}")

    def summary(model, content):
        content_token_length = calculate_token_length(content)
        max_token_length = 4096
        if content_token_length <= max_token_length:
            return predict(model, "请您帮忙总结下面的内容，尽可能不损失对投资有用的信息。\n" + content)
        else:
            part_size = content_token_length // 2  # Determine size for each part
            midpoint = len(content) // 2  # Find the middle point in characters
            part1 = content[:midpoint]
            part2 = content[midpoint:]
            while calculate_token_length(part1) > max_token_length or calculate_token_length(part2) > max_token_length:
                midpoint //= 2
                part1 = content[:midpoint]
                part2 = content[midpoint:]

            summary1 = summary(model, part1)
            summary2 = summary(model, part2)
            combined_summary = summary1 + summary2
            return predict(model, "请您帮忙总结下面的内容，尽可能不损失对投资有用的信息。\n" + combined_summary)

    idx, row_copy = row
    content = row_copy['content']
    return {
        "ann_date": row_copy['ann_date'],
        "rec_time": row_copy['rec_time'],
        "title": row_copy['title'],
        "name": row_copy['name'],
        "content": content,
        "summary": summary("deepseek-chat", content)

    }


def generate_dataset(data, max_samples=1000):
    global igpt
    igpt = OpenAIGPT(keys_path="apikeys.txt")
    results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(process_row, row) for row in data[:max_samples].iterrows()]
        for future in tqdm(futures, total=len(futures), desc="Processing rows"):
            result = future.result()
            if result:
                results.append(result)

    return results

In [6]:
result = generate_dataset(data, 1000)

Processing rows:   1%|          | 3/409 [00:18<40:17,  5.95s/it]

In [ ]:
result

[{'ann_date': '20240412',
  'rec_time': '2024-04-11 15:53:02',
  'title': '三一重工股份有限公司2024年第二次临时股东大会会议资料',
  'name': '三一重工',
  'content': '时间：2024-04-11 15:53:02\n标题：三一重工股份有限公司2024年第二次临时股东大会会议资料\n正文：2024 年第二次临时股东大会会议议案\n三一重工股份有限公司\n2024 年 4 月 19 日目录\n1、关于《2024 年员工持股计划（草案）及其摘要》的议案 .................2\n2、关于《2024 年员工持股计划管理办法》的议案 ................................3\n3、关于提请股东大会授权董事会办理公司 2024 年员工持股计划相关事宜\n的议案 .......................................................................................................4\n―1―议案一：\n三一重工股份有限公司\n关于《2024 年员工持股计划（草案）及其摘要》\n的议案\n各位股东及股东代表：\n为进一步完善公司法人治理结构，促进公司建立、健全激励约束\n机制，充分调动公司董事、监事、高级管理人员及员工的积极性和创\n造性，有效地将股东利益、公司利益和员工个人利益结合在一起，使\n各方共同关注公司的长远发展，根据《公司法》、《证券法》、《关于上\n市公司实施员工持股计划试点的指导意见》以及《公司章程》等规定，\n公司制定了《2024 年员工持股计划（草案）》及其摘要。\n本员工持股计划的参加对象共计不超过 6,972 人，所有参加对象\n均需在公司或公司全资或控股子公司任职；员工持股计划的设立规模\n不超过 591,688,244 元。\n具体内容详见 2024 年 4 月 4 日公司在上海证券交易所网站公开\n披露的公司《2024 年员工持股计划（草案）》及其摘要。\n以上议案，请予以审议。\n三一重工股份有限公司\n董 事 会\n2024 年 4 月 19 日\n―2―议案二：\n三一重工股份有限公司\n关于《2024 年员工持股

In [ ]:
df_result = pd.DataFrame(result)

In [ ]:
df_result.to_parquet('output.parquet')

In [ ]:
test = pd.read_parquet('output.parquet')

In [ ]:
print(test['summary'])

0     内容总结：\n三一重工股份有限公司在2024年第二次临时股东大会上提出了关于员工持股计划的议...
1     内容总结：\n本文是湖南启元律师事务所关于三一重工股份有限公司2024年员工持股计划的法律意...
2     2024年4月19日，三一重工股份有限公司将召开第二次临时股东大会。股东大会将采用现场投票和...
3     内容总结：\n1. 三一重工股份有限公司第八届监事会第十七次会议于2024年4月2日召开，共...
4     内容总结：\n本文介绍了三一重工股份有限公司可持续发展委员会工作细则，包括总则、人员组成、职...
5     内容总结：\n本文介绍了三一重工股份有限公司反商业贿赂制度的相关内容。主要包括总则、管理机构...
6     内容总结：\n2024年4月2日，三一重工股份有限公司召开董事会和监事会，通过了终止境外发行...
7     2024年三一重工员工持股计划草案摘要：\n\n- 目的是完善公司治理结构，激励员工，促进公...
8     这段内容主要是关于员工持股计划的管理和实施程序。其中包括了持有人会议的召集程序、持有人会议的...
9     内容总结：\n三一重工股份有限公司在2024年制定了员工持股计划管理办法，旨在完善公司的法人...
10    内容总结：\n1. 三一重工股份有限公司第八届董事会第十八次会议决议公告\n2. 通过制定可...
11    内容总结：三一重工股份有限公司于2024年4月2日召开职工代表大会，就公司拟实施的2024年...
12    内容总结：\n1. 三一重工股份有限公司通过集中竞价交易方式回购公司股份的进展公告。\n2....
13    内容总结：\n1. 三一重工股份有限公司通过集中竞价交易方式首次回购股份，回购股份占总股本比...
14    内容总结：\n1. 三一重工股份有限公司计划以集中竞价交易方式回购公司股份，回购资金总额不低...
15    内容总结：三一重工股份有限公司公告证券事务代表周利凯先生因工作变动原因辞职，辞职报告已经董事...
16    内容总结：\n2024年2月4日，三一重工股份有限公司董事会通过了回购股份的议案。根据相关规...
17    2024年2月5日，三一重工股份有限公司发布了关于以集中竞价交易方式回购公司股份方案

In [1]:
import pandas as pd

s = pd.read_parquet('/home/zhangmin/toby/Quant-GPT/data/announcement/600031.SH_summary.parquet')

In [13]:
s

,ann_date,ts_code,name,title,url,rec_time,content,content_token_len,summary,summary_token_len
0,20240412,600031.SH,三一重工,三一重工股份有限公司2024年第二次临时股东大会会议资料,http://dataclouds.cninfo.com.cn/shgonggao/2024...,2024-04-11 15:53:02,时间：2024-04-11 15:53:02\n标题：三一重工股份有限公司2024年第二次临...,916,时间：2024-04-11 15:53:02\n标题：三一重工股份有限公司2024年第二次临...,183
1,20240412,600031.SH,三一重工,湖南启元律师事务所关于三一重工股份有限公司2024年员工持股计划的法律意见书,http://dataclouds.cninfo.com.cn/shgonggao/2024...,2024-04-11 15:53:02,时间：2024-04-11 15:53:02\n标题：湖南启元律师事务所关于三一重工股份有限...,3302,时间：2024-04-11 15:53:02\n标题：湖南启元律师事务所关于三一重工股份有限...,109
2,20240404,600031.SH,三一重工,三一重工股份有限公司关于召开2024年第二次临时股东大会的通知,http://dataclouds.cninfo.com.cn/shgonggao/2024...,2024-04-03 18:07:03,时间：2024-04-03 18:07:03\n标题：三一重工股份有限公司关于召开2024年...,1668,时间：2024-04-03 18:07:03\n标题：三一重工股份有限公司关于召开2024年...,378
3,20240404,600031.SH,三一重工,三一重工股份有限公司第八届监事会第十七次会议决议公告,http://dataclouds.cninfo.com.cn/shgonggao/2024...,2024-04-03 18:07:03,时间：2024-04-03 18:07:03\n标题：三一重工股份有限公司第八届监事会第十七...,665,时间：2024-04-03 18:07:03\n标题：三一重工股份有限公司第八届监事会第十七...,191
4,20240404,600031.SH,三一重工,三一重工股份有限公司可持续发展委员会工作细则,http://dataclouds.cninfo.com.cn/shgonggao/2024...,2024-04-03 18:07:03,时间：2024-04-03 18:07:03\n标题：三一重工股份有限公司可持续发展委员会工...,1004,时间：2024-04-03 18:07:03\n标题：三一重工股份有限公司可持续发展委员会工...,232
...,...,...,...,...,...,...,...,...,...,...
565,20181224,600031.SH,三一重工,三一重工：关于收到政府补助的公告,http://static.cninfo.com.cn/finalpage/2018-12-...,2018-12-23 17:42:47,时间：2018-12-23 17:42:47\n标题：三一重工：关于收到政府补助的公告\n正...,1228,时间：2018-12-23 17:42:47\n标题：三一重工：关于收到政府补助的公告\n\...,813
566,20181224,600031.SH,三一重工,三一重工：关于使用自有闲置资金购买理财产品的进展公告,http://static.cninfo.com.cn/finalpage/2018-12-...,2018-12-23 17:42:47,时间：2018-12-23 17:42:47\n标题：三一重工：关于使用自有闲置资金购买理财...,1780,时间：2018-12-23 17:42:47\n标题：三一重工：关于使用自有闲置资金购买理财...,267
567,20181207,600031.SH,三一重工,三一重工：2018年第二次临时股东大会会议资料,http://static.cninfo.com.cn/finalpage/2018-12-...,2018-12-06 18:13:10,时间：2018-12-06 18:13:10\n标题：三一重工：2018年第二次临时股东大会...,4295,时间：2018-12-06 18:13:10\n标题：三一重工：2018年第二次临时股东大会...,284
568,20181024,600031.SH,三一重工,三一重工：2018年前三季度业绩预增公告,http://static.cninfo.com.cn/finalpage/2018-10-...,2018-10-23 17:48:35,时间：2018-10-23 17:48:35\n标题：三一重工：2018年前三季度业绩预增公...,697,时间：2018-10-23 17:48:35\n标题：三一重工：2018年前三季度业绩预增公...,204


In [12]:
print(s['summary'][565])

时间：2018-12-23 17:42:47
标题：三一重工：关于收到政府补助的公告

三一重工股份有限公司（以下简称“公司”）及其子公司于2018年2月5日至2018年12月20日收到与收益相关的政府补助款项共计225,886,776元。具体明细如下：

| 收到补助时间 | 获得补助单位 | 发放主体 | 补贴项目 | 补贴金额(元) |
| --- | --- | --- | --- | --- |
| 2018-4-12 | 上海三一重机股份有限公司 | 上海临港奉贤企业服务有限公司 | 财政扶持资金 | 3,826,763 |
| 2018-5-3 | 湖南三一众创孵化器有限公司 | 长沙经济开发区管理委员会 | 三一众智新城专项资金 | 4,225,000 |
| 2018-9-7 | 上海华兴数字科技有限公司 | 上海市浦东新区贸易发展推进中心 | 川沙园区政府扶持款 | 2,020,000 |
| 2018-11-2 | 三一汽车制造有限公司涟源分公司 | 湖南涟源经济开发区管理委员会 | 企业经营业务财税贡献奖励 | 10,000,000 |
| 2018-5-17 | 上海三一重机股份有限公司 | 国网上海市电力公司 | 光伏补贴款 | 2,739,777 |
| 2018-7-9 | 浙江三一装备有限公司 | 湖州市吴兴区财政局 | 财政补助 | 3,027,310 |
| 2018-7-12 | 上海三一重机股份有限公司 | 上海临港奉贤企业服务有限公司 | 财政扶持资金 | 4,244,455 |
| 2018-7-17 | 湖南三一众创孵化器有限公司 | 长沙县财政局 | 小微企业创业创新示范创客空间扶持资金 | 2,140,000 |
| 2018-7-20 | 三一汽车制造有限公司 | 长沙县地方税务局 | 税收减免 | 13,000,000 |
| 2018-7-20 | 三一汽车制造有限公司 | 长沙县地方税务局 | 税收减免 | 12,000,000 |
| 2018-8-1 | 湖南三一中阳机械有限公司 | 益阳市财政局高新区分局、益阳高新区经济合作局 | 财源建设专项资金 | 7,280,000 |
| 2018-9-28 | 三一汽车起重机械有限公司 | 宁乡高新技术产业园区管理委员会 | 产业扶持资金 | 5,440,0